In [2]:
import pandas as pd
import numpy as np

# Load data

In [3]:
train_data = pd.read_csv('data/train_v2.csv')

# Data pipeline

In [6]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    
])

ValueError: not enough values to unpack (expected 2, got 0)